# 0. Install Metavision SDK or OpenEB

Follow this webpage to install Metavision SDK or OpenEB:

https://docs.prophesee.ai/stable/installation/index.html

# 1. delete all .bias files (cleaning) - optional

In [1]:
# when data is recorded from Prophesee event camera you get the bias file with all the biases information.
# we are deleting it just so we can only have useful .raw files in folder.

# if you are not recording data from event-camera then avoid this step.

# just provide the folder path and it will delete all files with extension .bias

import os

# Specify the folder containing the .bias files
folder_path = '/home/aitsam/openeb/training_data/all_data/200_samples_all_classes/armroll'

# Get a list of all .bias files in the folder
bias_files = [file for file in os.listdir(folder_path) if file.endswith('.bias')]

for bias_file in bias_files:
    file_path = os.path.join(folder_path, bias_file)

    # Delete the file
    try:
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

# 2. Convert .raw to .avi (preprocessing)

In [4]:
# Don't forget to change 'folder_path' and 'convert_script_path'
# you will find this file here 'metavision/designer/core/samples/python/raw_to_video.py'


import os
import subprocess

def convert_raw_files(folder_path, conversion_script_path):
    # Iterate over all files and subfolders in the specified folder
    for root, dirs, files in os.walk(folder_path):
        # Iterate over each file in the current folder
        for file in files:
            if file.endswith(".raw"):
                raw_file_path = os.path.join(root, file)

                # Construct the output AVI file path
                avi_file_path = os.path.splitext(raw_file_path)[0] + ".avi"

                # Construct the command to run the conversion script
                command = f"python3 {conversion_script_path} -i {raw_file_path} -o {avi_file_path}"

                # Execute the command using subprocess
                subprocess.run(command, shell=True)

# Specify the main folder path and conversion script path
folder_path = '~/Downloads/input_data/N_light'
conversion_script_path = "~/openeb/metavision_all/metavision/designer/core/samples/python/raw_to_video.py"

# Call the function to convert raw files to AVI recursively
convert_raw_files(folder_path, conversion_script_path)


[HAL][WARNING] dlopen error: libmetavision_hal.so.4: cannot open shared object file: No such file or directory 
[HAL][WARNING] dlopen error: libmetavision_psee_hw_layer.so: cannot open shared object file: No such file or directory 


Generating file /home/aitsam/Downloads/input_data/N_light/normal_armroll2.avi


[HAL][WARNING] dlopen error: libmetavision_hal.so.4: cannot open shared object file: No such file or directory 
[HAL][WARNING] dlopen error: libmetavision_psee_hw_layer.so: cannot open shared object file: No such file or directory 


Generating file /home/aitsam/Downloads/input_data/N_light/normal_clap3.avi


[HAL][WARNING] dlopen error: libmetavision_hal.so.4: cannot open shared object file: No such file or directory 
[HAL][WARNING] dlopen error: libmetavision_psee_hw_layer.so: cannot open shared object file: No such file or directory 


Generating file /home/aitsam/Downloads/input_data/N_light/normal_fist2.avi


[HAL][WARNING] dlopen error: libmetavision_hal.so.4: cannot open shared object file: No such file or directory 
[HAL][WARNING] dlopen error: libmetavision_psee_hw_layer.so: cannot open shared object file: No such file or directory 


Generating file /home/aitsam/Downloads/input_data/N_light/normal_point2.avi


[HAL][WARNING] dlopen error: libmetavision_hal.so.4: cannot open shared object file: No such file or directory 
[HAL][WARNING] dlopen error: libmetavision_psee_hw_layer.so: cannot open shared object file: No such file or directory 


Generating file /home/aitsam/Downloads/input_data/N_light/Normal_wave2.avi


# 3. Data Labeling 

In [2]:
# This script will play .avi video one by one for labeling

# Don't forget to change 'folder_path' and 'label_tracking_script'
# you will find this labeling folder in Prophesee Knowledge center (https://support.prophesee.ai/portal/en/kb/articles/test-machine-learning-labeling-tool)

# A detailed annotating instruction is shown on the FLY. Here, we list some basic functionalities provided by the tool:
# hold left click and move the mouse on the scene image to draw a bbox (green)
# left click on an existing bbox to select it for editing. Once selected, the bbox will turn blue. 
# hold left click on one of the corners or borders of an existing bbox to resize it
# while the bbox is selected (in blue), you can press a number key to change the object id.  Press the C key to switch between class id and object id. Tab Enter key to finish the id modification. 
# R key to go to next frame. 
# E key to go to previous frame.
# D key to delete the selected bbox


import os
import subprocess

# Specify the folder containing the .avi files
folder_path = '/home/aitsam/Downloads/input_data'

# Get a list of all .avi files in the folder
avi_files = [file for file in os.listdir(folder_path) if file.endswith('.avi')]

# Path to the label_tracking.py script
label_tracking_script = '/home/aitsam/openeb/metavision_all/metavision/sdk/ml/python_samples/labelling_tools/label_tracking.py'

k=0
# Loop through each .avi file and run the label_tracking.py script
for avi_file in avi_files:
    video_path = os.path.join(folder_path, avi_file)
    
    print(k)
    k=+1
    
    # Set the frame rate for the video (you can adjust this value)
    fps = 100  # Example frame rate
    
    # Construct the command to run label_tracking.py with the appropriate arguments
    command = [
        'python3', label_tracking_script,
        '-i', video_path,
        '-f', str(fps)
    ]
    
    # Run the command
    try:
        subprocess.run(command, check=True)
        print(f"Finished processing: {video_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error processing {video_path}: {e}")

0


--- Image Legend ---


- Green: Manually set bbox
- Blue: BBox selected for edition or being edited
- [object_id, class_id]: Id couple on top left of a bounding box


--- Functionalities ---


- Hold left click and move the mouse on the scene image to draw a bbox.
- Left click on an existing bbox to select it for editing
- Hold left click on one of the corner an existing bbox to resize it
- Hold left click on one of the border an existing bbox to resize it
- Hold left click on an existing bbox then move the mouse to slide it
- Right click on an existing bbox to delete it
- Tab key to select next bbox for edition
- CTRL + Tab key to select previous bbox for edition
- D key to delete the selected bbox
- U key to delete all bbox in the sequence with the same object id as the selected bbox
- O key to overwrite all the future bbox with the same object id as the selected bbox.
- S key to stop tracking the selected bbox
- H key to hide bboxes
- V key to activate cursor visor
- Esc key to 

Traceback (most recent call last):
  File "/home/aitsam/openeb/metavision_all/metavision/sdk/ml/python_samples/labelling_tools/label_tracking.py", line 410, in <module>
    main(args)
  File "/home/aitsam/openeb/metavision_all/metavision/sdk/ml/python_samples/labelling_tools/label_tracking.py", line 404, in main
    bboxes_array = bboxstr2array(lines)
  File "/home/aitsam/openeb/metavision_all/metavision/sdk/ml/python_samples/labelling_tools/bbox_txt2npy.py", line 86, in bboxstr2array
    boxes_array = np.concatenate(list_boxes)
  File "<__array_function__ internals>", line 180, in concatenate
ValueError: need at least one array to concatenate




--- Image Legend ---


- Green: Manually set bbox
- Blue: BBox selected for edition or being edited
- [object_id, class_id]: Id couple on top left of a bounding box


--- Functionalities ---


- Hold left click and move the mouse on the scene image to draw a bbox.
- Left click on an existing bbox to select it for editing
- Hold left click on one of the corner an existing bbox to resize it
- Hold left click on one of the border an existing bbox to resize it
- Hold left click on an existing bbox then move the mouse to slide it
- Right click on an existing bbox to delete it
- Tab key to select next bbox for edition
- CTRL + Tab key to select previous bbox for edition
- D key to delete the selected bbox
- U key to delete all bbox in the sequence with the same object id as the selected bbox
- O key to overwrite all the future bbox with the same object id as the selected bbox.
- S key to stop tracking the selected bbox
- H key to hide bboxes
- V key to activate cursor visor
- Esc key to un

Traceback (most recent call last):
  File "/home/aitsam/openeb/metavision_all/metavision/sdk/ml/python_samples/labelling_tools/label_tracking.py", line 410, in <module>
    main(args)
  File "/home/aitsam/openeb/metavision_all/metavision/sdk/ml/python_samples/labelling_tools/label_tracking.py", line 404, in main
    bboxes_array = bboxstr2array(lines)
  File "/home/aitsam/openeb/metavision_all/metavision/sdk/ml/python_samples/labelling_tools/bbox_txt2npy.py", line 86, in bboxstr2array
    boxes_array = np.concatenate(list_boxes)
  File "<__array_function__ internals>", line 180, in concatenate
ValueError: need at least one array to concatenate




--- Image Legend ---


- Green: Manually set bbox
- Blue: BBox selected for edition or being edited
- [object_id, class_id]: Id couple on top left of a bounding box


--- Functionalities ---


- Hold left click and move the mouse on the scene image to draw a bbox.
- Left click on an existing bbox to select it for editing
- Hold left click on one of the corner an existing bbox to resize it
- Hold left click on one of the border an existing bbox to resize it
- Hold left click on an existing bbox then move the mouse to slide it
- Right click on an existing bbox to delete it
- Tab key to select next bbox for edition
- CTRL + Tab key to select previous bbox for edition
- D key to delete the selected bbox
- U key to delete all bbox in the sequence with the same object id as the selected bbox
- O key to overwrite all the future bbox with the same object id as the selected bbox.
- S key to stop tracking the selected bbox
- H key to hide bboxes
- V key to activate cursor visor
- Esc key to un

Traceback (most recent call last):
  File "/home/aitsam/openeb/metavision_all/metavision/sdk/ml/python_samples/labelling_tools/label_tracking.py", line 410, in <module>
    main(args)
  File "/home/aitsam/openeb/metavision_all/metavision/sdk/ml/python_samples/labelling_tools/label_tracking.py", line 292, in main
    retCode = cv2.waitKeyEx(autoplay_speed)
KeyboardInterrupt


KeyboardInterrupt: 

# 4. Convert .raw to .h5

In [5]:
# Don't forget to change the 'folder_path' and 'generate_h5_script' path.
# you will find this file here 'metavision/sdk/ml/python_samples/generate_hdf5/generate_hdf5.py'


import os
import subprocess

def generate_h5_files(folder_path, generate_h5_script):
    # List all .raw files in the folder
    raw_files = [file for file in os.listdir(folder_path) if file.endswith(".raw")]

    # Iterate over each .raw file
    for raw_file in raw_files:
        raw_file_path = os.path.join(folder_path, raw_file)
        print(raw_file_path)

        # Construct the output folder path
        output_folder = os.path.dirname(raw_file_path)
        #output_folder = '/home/aitsam/Documents/openeb/prophesee_gesture_dataset/point_left'
        

        # Construct the command to generate the HDF5 file
        command = f"python3 {generate_h5_script} {raw_file_path} -o {output_folder}"

        # Execute the command using subprocess
        subprocess.run(command, shell=True)

# Specify the folder path containing the raw files
#folder_path = '/home/aitsam/Documents/phd_code/data_processing_pipeline/DVS128Gesture/csvRAW_dvsGesture_160x120/raw_files/3'
folder_path = '/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/0'

# Specify the path to the generate_hdf5.py script
generate_h5_script = "/home/aitsam/openeb/metavision_all/metavision/sdk/ml/python_samples/generate_hdf5/generate_hdf5.py"

# Call the function to generate HDF5 files for the raw files
generate_h5_files(folder_path, generate_h5_script)


/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/0/user05_led_0.raw


[HAL][WARNING] dlopen error: libmetavision_hal.so.4: cannot open shared object file: No such file or directory 
[HAL][WARNING] dlopen error: libmetavision_psee_hw_layer.so: cannot open shared object file: No such file or directory 

0it [00:00, ?it/s]

Processing user05_led_0.raw, storing results in /home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/0/user05_led_0.h5



1it [00:00,  2.86it/s]
7it [00:00, 18.71it/s]
13it [00:00, 29.17it/s]
19it [00:00, 36.25it/s]
25it [00:00, 41.24it/s]
31it [00:00, 44.92it/s]
37it [00:01, 47.46it/s]
43it [00:01, 49.23it/s]
49it [00:01, 50.03it/s]
55it [00:01, 51.06it/s]
61it [00:01, 51.68it/s]
67it [00:01, 52.26it/s]
73it [00:01, 52.13it/s]
79it [00:01, 52.60it/s]
85it [00:01, 52.97it/s]
91it [00:02, 52.88it/s]
97it [00:02, 52.81it/s]
103it [00:02, 52.99it/s]
109it [00:02, 53.20it/s]
115it [00:02, 53.21it/s]
121it [00:02, 53.01it/s]
127it [00:02, 52.88it/s]
133it [00:02, 52.87it/s]
139it [00:02, 53.17it/s]
145it [00:03, 53.09it/s]
151it [00:03, 53.05it/s]
157it [00:03, 52.67it/s]
163it [00:03, 52.90it/s]
169it [00:03, 52.98it/s]
175it [00:03, 52.93it/s]
181it [00:03, 52.59it/s]
187it [00:03, 52.80it/s]
193it [00:03, 52.77it/s]
199it [00:04, 53.07it/s]
205it [00:04, 52.75it/s]
211it [00:04, 51.23it/s]
217it [00:04, 51.66it/s]
223it [00:04, 51.96it/s]
229it [00:04, 51.64it/s]
235it [00:04, 51.07it/s]
241it [00:04, 51.2

/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/0/user25_led_0.raw


...


1it [00:00,  2.28it/s]
6it [00:00, 13.36it/s]
11it [00:00, 21.76it/s]
16it [00:00, 27.98it/s]
21it [00:00, 32.67it/s]
26it [00:01, 36.36it/s]
31it [00:01, 38.95it/s]
36it [00:01, 40.69it/s]
41it [00:01, 42.22it/s]
46it [00:01, 42.92it/s]
51it [00:01, 43.44it/s]
56it [00:01, 44.29it/s]
61it [00:01, 44.86it/s]
66it [00:01, 45.18it/s]
71it [00:01, 45.50it/s]
76it [00:02, 45.55it/s]
81it [00:02, 45.77it/s]
86it [00:02, 45.52it/s]
91it [00:02, 45.29it/s]
96it [00:02, 45.03it/s]
101it [00:02, 45.29it/s]
106it [00:02, 45.20it/s]
111it [00:02, 45.57it/s]
116it [00:02, 45.29it/s]
121it [00:03, 45.23it/s]
126it [00:03, 45.64it/s]
131it [00:03, 45.66it/s]
136it [00:03, 45.54it/s]
141it [00:03, 45.31it/s]
146it [00:03, 45.20it/s]
151it [00:03, 45.36it/s]
156it [00:03, 45.34it/s]
161it [00:03, 45.41it/s]
166it [00:04, 45.58it/s]
171it [00:04, 45.86it/s]
176it [00:04, 45.74it/s]
181it [00:04, 45.68it/s]
186it [00:04, 45.68it/s]
191it [00:04, 45.73it/s]
196it [00:04, 45.83it/s]
201it [00:04, 45.78it

/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/0/user22_natural_0.raw


[HAL][WARNING] dlopen error: libmetavision_hal.so.4: cannot open shared object file: No such file or directory 
[HAL][WARNING] dlopen error: libmetavision_psee_hw_layer.so: cannot open shared object file: No such file or directory 

0it [00:00, ?it/s]

Processing user22_natural_0.raw, storing results in /home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/0/user22_natural_0.h5



1it [00:00,  2.23it/s]
6it [00:00, 13.22it/s]
11it [00:00, 21.50it/s]
16it [00:00, 27.75it/s]
21it [00:00, 32.50it/s]
26it [00:01, 35.96it/s]
31it [00:01, 38.69it/s]
36it [00:01, 40.76it/s]
41it [00:01, 42.21it/s]
46it [00:01, 43.21it/s]
51it [00:01, 44.17it/s]
56it [00:01, 44.61it/s]
61it [00:01, 44.97it/s]
66it [00:01, 44.83it/s]
71it [00:02, 44.96it/s]
76it [00:02, 45.20it/s]
81it [00:02, 45.50it/s]
86it [00:02, 45.48it/s]
91it [00:02, 45.22it/s]
96it [00:02, 45.36it/s]
101it [00:02, 45.61it/s]
106it [00:02, 45.64it/s]
111it [00:02, 45.60it/s]
116it [00:02, 45.67it/s]
121it [00:03, 45.51it/s]
126it [00:03, 45.57it/s]
131it [00:03, 45.79it/s]
136it [00:03, 45.72it/s]
141it [00:03, 45.80it/s]
146it [00:03, 44.74it/s]
151it [00:03, 44.28it/s]
156it [00:03, 44.21it/s]
161it [00:03, 44.40it/s]
166it [00:04, 44.62it/s]
171it [00:04, 44.91it/s]
176it [00:04, 44.93it/s]
181it [00:04, 44.90it/s]
186it [00:04, 45.03it/s]
191it [00:04, 45.29it/s]
196it [00:04, 45.43it/s]
201it [00:04, 45.53it

# 5. fetch .h5 and .npy files

In [39]:
# for training we need .h5 file and its corresponding _bbox.npy file.
# we created separate folder to fetcht these files.
# Don't forget to update 'source_folder' and 'destination_folder'



import os
import shutil

def copy_npy_h5_files(source_folder, destination_folder):
    # Ensure the destination folder exists
    os.makedirs(destination_folder, exist_ok=True)

    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith('.npy') or file.endswith('.h5'):
                source_path = os.path.join(root, file)
                destination_path = os.path.join(destination_folder, file)
                shutil.copy(source_path, destination_path)

if __name__ == "__main__":
    source_folder = "/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/10"
    destination_folder = "/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/10_h5"
    copy_npy_h5_files(source_folder, destination_folder)


# 6. train test split

In [51]:
import os
import random
import shutil

def split_files(input_folder, train_folder, test_folder, val_folder, train_ratio, test_ratio):
    # Step 1: Create folders if they don't exist
    for folder in [train_folder, test_folder, val_folder]:
        if not os.path.exists(folder):
            os.makedirs(folder)

    # Step 2: Load the list of files
    h5_files = [f for f in os.listdir(input_folder) if f.endswith('.h5')]

    # Step 3: Shuffle the list of files
    random.shuffle(h5_files)

    # Step 4: Split the files
    num_files = len(h5_files)
    train_cutoff = int(num_files * train_ratio)
    test_cutoff = int(num_files * (train_ratio + test_ratio))

    train_files = h5_files[:train_cutoff]
    test_files = h5_files[train_cutoff:test_cutoff]
    val_files = h5_files[test_cutoff:]

    # Step 5: Copy the files to their respective folders along with their corresponding .npy files
    for h5_file in train_files:
        h5_path = os.path.join(input_folder, h5_file)
        npy_file = h5_file.replace('.h5', '_bbox.npy')
        npy_path = os.path.join(input_folder, npy_file)
        shutil.copy(h5_path, os.path.join(train_folder, h5_file))
        shutil.copy(npy_path, os.path.join(train_folder, npy_file))

    for h5_file in test_files:
        h5_path = os.path.join(input_folder, h5_file)
        npy_file = h5_file.replace('.h5', '_bbox.npy')
        npy_path = os.path.join(input_folder, npy_file)
        shutil.copy(h5_path, os.path.join(test_folder, h5_file))
        shutil.copy(npy_path, os.path.join(test_folder, npy_file))

    for h5_file in val_files:
        h5_path = os.path.join(input_folder, h5_file)
        npy_file = h5_file.replace('.h5', '_bbox.npy')
        npy_path = os.path.join(input_folder, npy_file)
        shutil.copy(h5_path, os.path.join(val_folder, h5_file))
        shutil.copy(npy_path, os.path.join(val_folder, npy_file))

if __name__ == "__main__":
    input_folder = "/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/10_h5"  # Replace with the path to your folder containing .h5 and .npy files
    train_folder = "/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/dvs_train_data/train"  # Replace with the path where you want to store train files
    test_folder = "/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/dvs_train_data/test"  # Replace with the path where you want to store test files
    val_folder = "/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/dvs_train_data/val"  # Replace with the path where you want to store validation files
    train_ratio = 0.7
    test_ratio = 0.1

    split_files(input_folder, train_folder, test_folder, val_folder, train_ratio, test_ratio)


# 7. Semi-Automatic labeling (3sec dataset)

In [45]:
#The label is by default set to EventBbox format,
# except that only the column “ts” and “class_id” are actually used, the rest can be set to a constant dummy value.

# If we have event data of same time duration then we can label only only input file and replicate it for rest.

# change path of 'folder1_path', 'destination_folder_path' and 'source_folder_path'.

import os
import shutil

# Step 1: List all files in the first folder and extract their names without extensions
folder1_path = '/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/0'
folder1_files = os.listdir(folder1_path)
folder1_file_names = [os.path.splitext(file)[0] for file in folder1_files]
print(len(folder1_file_names))

# Specify the folder where you want to create the copies
destination_folder_path = '/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/0'

# Specify the folder containing the files you want to copy
source_folder_path = '/home/aitsam/openeb/training_data/DVS128/csvRAW_dvsGesture/raw_files/0_sample'

# Get the list of files in the source folder
source_files = os.listdir(source_folder_path)

# Ensure the destination folder exists, create it if necessary
os.makedirs(destination_folder_path, exist_ok=True)

# Create 150 copies of each file in the destination folder with names from folder1_file_names
folder1_file_names = folder1_file_names

for source_file in source_files:
    source_file_path = os.path.join(source_folder_path, source_file)
    
    # Copy the file 150 times
    for name in folder1_file_names:
        destination_file_name = name + os.path.splitext(source_file)[1]
        destination_file_path = os.path.join(destination_folder_path, destination_file_name)
        shutil.copyfile(source_file_path, destination_file_path)


        # Rename the copied files in the destination folder
for name in folder1_file_names:
    for source_file in source_files:
        # Construct the old and new file paths
        old_file_path = os.path.join(destination_folder_path, name + os.path.splitext(source_file)[1])
        new_file_path = os.path.join(destination_folder_path, name + '_bbox' + os.path.splitext(source_file)[1])
        
        # Rename the file
        os.rename(old_file_path, new_file_path)

# Print a message to confirm that the files have been renamed
print("Files have been renamed in the destination folder.")

121
Files have been renamed in the destination folder.
